In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max.columns', 100)
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

Считаем данные и посмотрим на первые несколько строк. Видим, что у нас тут немало категориальных признаков.

In [3]:
df = pd.read_csv('bank.csv')

In [4]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,0
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,0
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,0
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,0
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 17 columns):
age          4521 non-null int64
job          4521 non-null object
marital      4521 non-null object
education    4521 non-null object
default      4521 non-null object
balance      4521 non-null int64
housing      4521 non-null object
loan         4521 non-null object
contact      4521 non-null object
day          4521 non-null int64
month        4521 non-null object
duration     4521 non-null int64
campaign     4521 non-null int64
pdays        4521 non-null int64
previous     4521 non-null int64
poutcome     4521 non-null object
y            4521 non-null int64
dtypes: int64(8), object(9)
memory usage: 600.5+ KB


Всего 9 признаков со строковыми значениями.

In [6]:
df.columns[df.dtypes == 'object']

Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'poutcome'],
      dtype='object')

## Без категориальных признаков
Попытаемся сначала просто проигнорировать категориальные признаки. Обучим случайный лес и посмотрим на ROC AUC на кросс-валидации и на отоженной выборке. Это будет наш бейзлайн. 

In [7]:
df_no_cat, y = df.loc[:, df.dtypes != 'object'].drop('y', axis=1), df['y']

In [8]:
df_no_cat_part, df_no_cat_valid, y_train_part, y_valid = train_test_split(df_no_cat, y,
                                                                          test_size=.3, 
                                                                          stratify=y, 
                                                                          random_state=17)

In [9]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

In [12]:
forest = RandomForestClassifier(n_estimators=100, random_state=17)

In [13]:
np.mean(cross_val_score(forest, df_no_cat_part, y_train_part, cv=skf, scoring='roc_auc'))

0.8510538239237666

In [14]:
forest.fit(df_no_cat_part, y_train_part)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=17, verbose=0, warm_start=False)

In [15]:
roc_auc_score(y_valid, forest.predict_proba(df_no_cat_valid)[:, 1])

0.8631508998911164

## LabelEncoder для категориальных признаков
Сделаем то же самое, но попробуем закодировать категориальные признаки по-простому: с помощью `LabelEncoder`.

In [16]:
from sklearn.preprocessing import LabelEncoder

In [17]:
label_encoder = LabelEncoder()

In [18]:
df_cat_label_enc = df.copy().drop('y', axis=1)
for col in df.columns[df.dtypes == 'object']:
    df_cat_label_enc[col] = label_encoder.fit_transform(df_cat_label_enc[col])

In [19]:
df_cat_label_enc.shape

(4521, 16)

In [20]:
df_cat_label_enc_part, df_cat_label_enc_valid = train_test_split(df_cat_label_enc, test_size=.3, 
                                                    stratify=y, random_state=17)

In [21]:
np.mean(cross_val_score(forest, df_cat_label_enc_part, y_train_part, cv=skf, scoring='roc_auc'))

0.8937578330398497

In [22]:
forest.fit(df_cat_label_enc_part, y_train_part)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=17, verbose=0, warm_start=False)

In [24]:
roc_auc_score(y_valid, forest.predict_proba(df_cat_label_enc_valid)[:, 1])

0.908855334230022

## Бинаризация категориальных признаков (dummies, OHE)
Теперь сделаем то, что обычно по умолчанию и делают – бинаризацию категориальных признаков. Dummy-признаки, One-Hot Encoding... с небольшими различиями это об одном же - для каждого значения каждого категориального признака завести свой бинарный признак.

In [25]:
df_cat_dummies = pd.get_dummies(df, columns=df.columns[df.dtypes == 'object']).drop('y', axis=1)

In [26]:
df_cat_dummies.shape

(4521, 51)

In [27]:
df_cat_dummies_part, df_cat_dummies_valid = train_test_split(df_cat_dummies, test_size=.3, 
                                                    stratify=y, random_state=17)

In [28]:
np.mean(cross_val_score(forest, df_cat_dummies_part, y_train_part, cv=skf, scoring='roc_auc'))

0.8970178304492578

In [29]:
forest.fit(df_cat_dummies_part, y_train_part)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=17, verbose=0, warm_start=False)

In [30]:
roc_auc_score(y_valid, forest.predict_proba(df_cat_dummies_valid)[:, 1])

0.9172511155233887

## Попарные взаимодействия признаков
Пока лес все еще лучше регрессии (хотя мы не тюнили гиперпараметры, но и не будем). Мы хотим идти дальше. Мощной техникой для работы с категориальными признаками будет учет попарных взаимодействий признаков (feature interactions). Построим попарные взаимодействия всех признаков. Вообще тут можно пойти дальше и строить взаимодействия трех и более признаков. Owen Zhang [как-то строил](https://www.youtube.com/watch?v=LgLcfZjNF44) даже 7-way interactions. Чего не сделаешь ради победы на Kaggle! :)

In [31]:
df_interact = df.copy()

In [32]:
cat_features = df.columns[df.dtypes == 'object']
for i, col1 in enumerate(cat_features):
    for j, col2 in enumerate(cat_features[i + 1:]):
        df_interact[col1 + '_' + col2] = df_interact[col1] + '_' + df_interact[col2] 

In [33]:
df_interact.shape

(4521, 53)

In [34]:
df_interact.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,job_marital,job_education,job_default,job_housing,job_loan,job_contact,job_month,job_poutcome,marital_education,marital_default,marital_housing,marital_loan,marital_contact,marital_month,marital_poutcome,education_default,education_housing,education_loan,education_contact,education_month,education_poutcome,default_housing,default_loan,default_contact,default_month,default_poutcome,housing_loan,housing_contact,housing_month,housing_poutcome,loan_contact,loan_month,loan_poutcome,contact_month,contact_poutcome,month_poutcome
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,0,unemployed_married,unemployed_primary,unemployed_no,unemployed_no,unemployed_no,unemployed_cellular,unemployed_oct,unemployed_unknown,married_primary,married_no,married_no,married_no,married_cellular,married_oct,married_unknown,primary_no,primary_no,primary_no,primary_cellular,primary_oct,primary_unknown,no_no,no_no,no_cellular,no_oct,no_unknown,no_no,no_cellular,no_oct,no_unknown,no_cellular,no_oct,no_unknown,cellular_oct,cellular_unknown,oct_unknown
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,0,services_married,services_secondary,services_no,services_yes,services_yes,services_cellular,services_may,services_failure,married_secondary,married_no,married_yes,married_yes,married_cellular,married_may,married_failure,secondary_no,secondary_yes,secondary_yes,secondary_cellular,secondary_may,secondary_failure,no_yes,no_yes,no_cellular,no_may,no_failure,yes_yes,yes_cellular,yes_may,yes_failure,yes_cellular,yes_may,yes_failure,cellular_may,cellular_failure,may_failure
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,0,management_single,management_tertiary,management_no,management_yes,management_no,management_cellular,management_apr,management_failure,single_tertiary,single_no,single_yes,single_no,single_cellular,single_apr,single_failure,tertiary_no,tertiary_yes,tertiary_no,tertiary_cellular,tertiary_apr,tertiary_failure,no_yes,no_no,no_cellular,no_apr,no_failure,yes_no,yes_cellular,yes_apr,yes_failure,no_cellular,no_apr,no_failure,cellular_apr,cellular_failure,apr_failure
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,0,management_married,management_tertiary,management_no,management_yes,management_yes,management_unknown,management_jun,management_unknown,married_tertiary,married_no,married_yes,married_yes,married_unknown,married_jun,married_unknown,tertiary_no,tertiary_yes,tertiary_yes,tertiary_unknown,tertiary_jun,tertiary_unknown,no_yes,no_yes,no_unknown,no_jun,no_unknown,yes_yes,yes_unknown,yes_jun,yes_unknown,yes_unknown,yes_jun,yes_unknown,unknown_jun,unknown_unknown,jun_unknown
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,0,blue-collar_married,blue-collar_secondary,blue-collar_no,blue-collar_yes,blue-collar_no,blue-collar_unknown,blue-collar_may,blue-collar_unknown,married_secondary,married_no,married_yes,married_no,married_unknown,married_may,married_unknown,secondary_no,secondary_yes,secondary_no,secondary_unknown,secondary_may,secondary_unknown,no_yes,no_no,no_unknown,no_may,no_unknown,yes_no,yes_unknown,yes_may,yes_unknown,no_unknown,no_may,no_unknown,unknown_may,unknown_unknown,may_unknown


## Бинаризация категориальных признаков (dummies, OHE) + попарные взаимодействия
Получилось аж 824 бинарных признака – многовато для такой задачи, и тут случайный лес начинает не справляться, да и логистическая регрессия сработала хуже, чем в прошлый раз.

In [35]:
df_interact_cat_dummies = pd.get_dummies(df_interact, columns=df_interact.columns[df_interact.dtypes == 'object']).drop('y', axis=1)

In [36]:
df_interact_cat_dummies.shape

(4521, 824)

In [37]:
df_interact_cat_dummies_part, df_interact_cat_dummies_valid = train_test_split(df_interact_cat_dummies, test_size=.3, 
                                                    stratify=y, random_state=17)

In [38]:
np.mean(cross_val_score(forest, df_interact_cat_dummies_part, y_train_part, cv=skf, scoring='roc_auc'))

0.836351772630046

In [39]:
forest.fit(df_interact_cat_dummies_part, y_train_part)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=17, verbose=0, warm_start=False)

In [40]:
roc_auc_score(y_valid, forest.predict_proba(df_interact_cat_dummies_valid)[:, 1])

0.8618112043382651

Случайному лесу уже тяжеловато, когда признаков так много, а вот логистической регрессии – норм.

In [73]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(solver='lbfgs', max_iter=500, random_state=17)

In [74]:
np.mean(cross_val_score(logit, df_interact_cat_dummies_part, y_train_part, cv=skf, scoring='roc_auc'))

0.8781691428991524

In [75]:
logit.fit(df_interact_cat_dummies_part, y_train_part)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=500, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=17, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [76]:
roc_auc_score(y_valid, logit.predict_proba(df_interact_cat_dummies_valid)[:, 1])

0.9004889088153034

## Mean Target
Теперь будем использовать технику кодирования категориальных признаков средним значением целевого признака. Это очень мощная техника, правда, надо умело ее использовать – легко переобучиться. 
Основная идея – для каждого значения категориального признака посчитать среднее значение целевого признака и заменить категориальный признак на посчитанные средние. Правда, считать средние надо на кросс-валидации, а то легко переобучиться. 
Но далее я адресую к видео топ-участников соревнований Kaggle, от них можно узнать про эту технику из первых уст. 
- [Специализация](https://www.coursera.org/specializations/aml) "Advanced Machine Learning" на Coursera, [курс](https://www.coursera.org/learn/competitive-data-science)", How to Win a Data Science Competition: Learn from Top Kagglers", несколько видео посвящено различным способам построяния признаков с задействованием целевого, и как при этом не переобучиться. Рассказывает Дмитрий Алтухов
- [Лекция](https://www.youtube.com/watch?v=g335THJxkto) с презентацией решения конкурса Kaggle BNP paribas, Станислав Семенов

Похожая техника [используется](https://tech.yandex.com/catboost/doc/dg/concepts/algorithm-main-stages_cat-to-numberic-docpage/) и в CatBoost.

Для начала давайте таким образом закодируем исходные категориальные признаки.

In [47]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [48]:
train_df, y = df.copy(), df['y']
train_df_part, valid_df, y_train_part, y_valid = train_test_split(train_df.drop('y', axis=1), y, 
                                                                  test_size=.3, stratify=y, 
                                                                               random_state=17)

In [49]:
def mean_target_enc(train_df, y_train, valid_df, skf):
    import warnings
    warnings.filterwarnings('ignore')
    
    glob_mean = y_train.mean()
    train_df = pd.concat([train_df, pd.Series(y_train, name='y')], axis=1)
    new_train_df = train_df.copy()
    
    cat_features = train_df.columns[train_df.dtypes == 'object'].tolist()    

    for col in cat_features:
        new_train_df[col + '_mean_target'] = [glob_mean for _ in range(new_train_df.shape[0])]

    for train_idx, valid_idx in skf.split(train_df, y_train):
        train_df_cv, valid_df_cv = train_df.iloc[train_idx, :], train_df.iloc[valid_idx, :]

        for col in cat_features:
            
            means = valid_df_cv[col].map(train_df_cv.groupby(col)['y'].mean())
            valid_df_cv[col + '_mean_target'] = means.fillna(glob_mean)
            
        new_train_df.iloc[valid_idx] = valid_df_cv
    
    new_train_df.drop(cat_features + ['y'], axis=1, inplace=True)
    
    for col in cat_features:
        means = valid_df[col].map(train_df.groupby(col)['y'].mean())
        valid_df[col + '_mean_target'] = means.fillna(glob_mean)
        
    valid_df.drop(train_df.columns[train_df.dtypes == 'object'], axis=1, inplace=True)
    
    return new_train_df, valid_df

In [50]:
train_mean_target_part, valid_mean_target = mean_target_enc(train_df_part, y_train_part, valid_df, skf)

In [51]:
np.mean(cross_val_score(forest, train_mean_target_part, y_train_part, cv=skf, scoring='roc_auc'))

0.8977973842022902

In [52]:
forest.fit(train_mean_target_part, y_train_part)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=17, verbose=0, warm_start=False)

In [53]:
roc_auc_score(y_valid, forest.predict_proba(valid_mean_target)[:, 1])

0.9187882960780546

## Mean Target + попарные взаимодействия

In [54]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [55]:
train_df, y = df_interact.drop('y', axis=1).copy(), df_interact['y']
train_df_part, valid_df, y_train_part, y_valid = train_test_split(train_df, y, 
                                                                  test_size=.3, stratify=y, 
                                                                  random_state=17)

In [56]:
train_mean_target_part, valid_mean_target = mean_target_enc(train_df_part, y_train_part, valid_df, skf)

In [57]:
np.mean(cross_val_score(forest, train_mean_target_part, y_train_part, cv=skf, scoring='roc_auc'))

0.8923825787942544

In [58]:
forest.fit(train_mean_target_part, y_train_part)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=17, verbose=0, warm_start=False)

In [59]:
roc_auc_score(y_valid, forest.predict_proba(valid_mean_target)[:, 1])

0.90931702213967

Опять лучше справляется логистическая регрессия.

In [77]:
np.mean(cross_val_score(logit, train_mean_target_part, y_train_part, cv=skf, scoring='roc_auc'))

0.8873066945797115

In [78]:
logit.fit(train_mean_target_part, y_train_part)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=500, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=17, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [79]:
roc_auc_score(y_valid, logit.predict_proba(valid_mean_target)[:, 1])

0.8986901940690452

## Catboost
В библиотеке [Catboost](https://catboost.yandex), помимо всего прочего, реализована как раз техника кодирования категориальных значений средним значением целевого признака. Результаты получаются хорошими именно когда в данных много важных категориальных признаков. Из минусов можно отметить меньшую (пока что) производительность в сравнении с Xgboost и LightGBM.

In [64]:
from catboost import CatBoostClassifier

In [65]:
ctb = CatBoostClassifier(random_seed=17)

In [66]:
train_df, y = df.drop('y', axis=1), df['y']
train_df_part, valid_df, y_train_part, y_valid = train_test_split(train_df, y, 
                                                                  test_size=.3, stratify=y, 
                                                                  random_state=17)

In [67]:
cat_features_idx = np.where(train_df_part.dtypes == 'object')[0].tolist()

In [68]:
%%time
cv_scores = []
for train_idx, test_idx in skf.split(train_df_part, y_train_part):
    cv_train_df, cv_valid_df = train_df_part.iloc[train_idx, :], train_df_part.iloc[test_idx, :]
    y_cv_train, y_cv_valid = y_train_part.iloc[train_idx], y_train_part.iloc[test_idx]
    
    ctb.fit(cv_train_df, y_cv_train,
        cat_features=cat_features_idx);
    
    cv_scores.append(roc_auc_score(y_cv_valid, ctb.predict_proba(cv_valid_df)[:, 1]))

Learning rate set to 0.02179
0:	learn: 0.6692629	total: 180ms	remaining: 2m 59s
1:	learn: 0.6458308	total: 398ms	remaining: 3m 18s
2:	learn: 0.6264219	total: 533ms	remaining: 2m 57s
3:	learn: 0.6055471	total: 640ms	remaining: 2m 39s
4:	learn: 0.5858460	total: 735ms	remaining: 2m 26s
5:	learn: 0.5676662	total: 874ms	remaining: 2m 24s
6:	learn: 0.5510922	total: 1.1s	remaining: 2m 36s
7:	learn: 0.5336998	total: 1.16s	remaining: 2m 24s
8:	learn: 0.5185215	total: 1.26s	remaining: 2m 19s
9:	learn: 0.5020085	total: 1.36s	remaining: 2m 14s
10:	learn: 0.4875820	total: 1.46s	remaining: 2m 11s
11:	learn: 0.4783670	total: 1.5s	remaining: 2m 3s
12:	learn: 0.4695063	total: 1.66s	remaining: 2m 6s
13:	learn: 0.4593504	total: 1.85s	remaining: 2m 10s
14:	learn: 0.4465721	total: 1.95s	remaining: 2m 7s
15:	learn: 0.4392295	total: 2.02s	remaining: 2m 4s
16:	learn: 0.4301279	total: 2.07s	remaining: 1m 59s
17:	learn: 0.4205782	total: 2.18s	remaining: 1m 58s
18:	learn: 0.4106982	total: 2.42s	remaining: 2m 5s


160:	learn: 0.2260951	total: 22.1s	remaining: 1m 55s
161:	learn: 0.2259698	total: 22.3s	remaining: 1m 55s
162:	learn: 0.2256186	total: 22.4s	remaining: 1m 55s
163:	learn: 0.2255874	total: 22.5s	remaining: 1m 54s
164:	learn: 0.2255657	total: 22.6s	remaining: 1m 54s
165:	learn: 0.2252885	total: 22.7s	remaining: 1m 53s
166:	learn: 0.2249981	total: 22.8s	remaining: 1m 53s
167:	learn: 0.2249149	total: 22.9s	remaining: 1m 53s
168:	learn: 0.2249130	total: 22.9s	remaining: 1m 52s
169:	learn: 0.2247894	total: 23s	remaining: 1m 52s
170:	learn: 0.2244786	total: 23.1s	remaining: 1m 52s
171:	learn: 0.2243104	total: 23.2s	remaining: 1m 51s
172:	learn: 0.2240629	total: 23.3s	remaining: 1m 51s
173:	learn: 0.2238092	total: 23.4s	remaining: 1m 51s
174:	learn: 0.2236208	total: 23.6s	remaining: 1m 51s
175:	learn: 0.2235556	total: 23.6s	remaining: 1m 50s
176:	learn: 0.2235494	total: 23.7s	remaining: 1m 50s
177:	learn: 0.2232734	total: 23.8s	remaining: 1m 49s
178:	learn: 0.2231677	total: 23.9s	remaining: 1m

321:	learn: 0.2050191	total: 41.6s	remaining: 1m 27s
322:	learn: 0.2050179	total: 41.7s	remaining: 1m 27s
323:	learn: 0.2049265	total: 41.8s	remaining: 1m 27s
324:	learn: 0.2047701	total: 41.9s	remaining: 1m 26s
325:	learn: 0.2046692	total: 41.9s	remaining: 1m 26s
326:	learn: 0.2046205	total: 42s	remaining: 1m 26s
327:	learn: 0.2045147	total: 42.2s	remaining: 1m 26s
328:	learn: 0.2044056	total: 42.4s	remaining: 1m 26s
329:	learn: 0.2043590	total: 42.4s	remaining: 1m 26s
330:	learn: 0.2042150	total: 42.5s	remaining: 1m 25s
331:	learn: 0.2040969	total: 42.6s	remaining: 1m 25s
332:	learn: 0.2039806	total: 42.8s	remaining: 1m 25s
333:	learn: 0.2039719	total: 42.9s	remaining: 1m 25s
334:	learn: 0.2038014	total: 43s	remaining: 1m 25s
335:	learn: 0.2035851	total: 43.2s	remaining: 1m 25s
336:	learn: 0.2035848	total: 43.3s	remaining: 1m 25s
337:	learn: 0.2034700	total: 43.4s	remaining: 1m 25s
338:	learn: 0.2034016	total: 43.6s	remaining: 1m 24s
339:	learn: 0.2033795	total: 43.7s	remaining: 1m 2

480:	learn: 0.1860431	total: 1m 1s	remaining: 1m 6s
481:	learn: 0.1858782	total: 1m 1s	remaining: 1m 6s
482:	learn: 0.1857056	total: 1m 1s	remaining: 1m 6s
483:	learn: 0.1855378	total: 1m 1s	remaining: 1m 5s
484:	learn: 0.1855201	total: 1m 1s	remaining: 1m 5s
485:	learn: 0.1853870	total: 1m 2s	remaining: 1m 5s
486:	learn: 0.1852599	total: 1m 2s	remaining: 1m 5s
487:	learn: 0.1850494	total: 1m 2s	remaining: 1m 5s
488:	learn: 0.1847174	total: 1m 2s	remaining: 1m 5s
489:	learn: 0.1846758	total: 1m 2s	remaining: 1m 5s
490:	learn: 0.1845558	total: 1m 2s	remaining: 1m 5s
491:	learn: 0.1844719	total: 1m 3s	remaining: 1m 5s
492:	learn: 0.1844237	total: 1m 3s	remaining: 1m 4s
493:	learn: 0.1843518	total: 1m 3s	remaining: 1m 4s
494:	learn: 0.1841603	total: 1m 3s	remaining: 1m 4s
495:	learn: 0.1841138	total: 1m 3s	remaining: 1m 4s
496:	learn: 0.1839975	total: 1m 3s	remaining: 1m 4s
497:	learn: 0.1838865	total: 1m 3s	remaining: 1m 4s
498:	learn: 0.1838047	total: 1m 3s	remaining: 1m 4s
499:	learn: 

639:	learn: 0.1686358	total: 1m 21s	remaining: 45.9s
640:	learn: 0.1685415	total: 1m 21s	remaining: 45.8s
641:	learn: 0.1684627	total: 1m 21s	remaining: 45.7s
642:	learn: 0.1683827	total: 1m 22s	remaining: 45.5s
643:	learn: 0.1682855	total: 1m 22s	remaining: 45.4s
644:	learn: 0.1682142	total: 1m 22s	remaining: 45.3s
645:	learn: 0.1681356	total: 1m 22s	remaining: 45.2s
646:	learn: 0.1681152	total: 1m 22s	remaining: 45s
647:	learn: 0.1680524	total: 1m 22s	remaining: 44.9s
648:	learn: 0.1680061	total: 1m 22s	remaining: 44.8s
649:	learn: 0.1678212	total: 1m 22s	remaining: 44.6s
650:	learn: 0.1677598	total: 1m 23s	remaining: 44.5s
651:	learn: 0.1676721	total: 1m 23s	remaining: 44.4s
652:	learn: 0.1675438	total: 1m 23s	remaining: 44.2s
653:	learn: 0.1673742	total: 1m 23s	remaining: 44.2s
654:	learn: 0.1672718	total: 1m 23s	remaining: 44s
655:	learn: 0.1672290	total: 1m 23s	remaining: 43.9s
656:	learn: 0.1670549	total: 1m 23s	remaining: 43.7s
657:	learn: 0.1669887	total: 1m 23s	remaining: 43.

...

308:	learn: 0.2117511	total: 32.7s	remaining: 1m 13s
309:	learn: 0.2117469	total: 32.8s	remaining: 1m 12s
310:	learn: 0.2117466	total: 32.8s	remaining: 1m 12s
311:	learn: 0.2116420	total: 32.9s	remaining: 1m 12s
312:	learn: 0.2116105	total: 32.9s	remaining: 1m 12s
313:	learn: 0.2114583	total: 33s	remaining: 1m 12s
314:	learn: 0.2112072	total: 33.2s	remaining: 1m 12s
315:	learn: 0.2108572	total: 33.4s	remaining: 1m 12s
316:	learn: 0.2107444	total: 33.4s	remaining: 1m 12s
317:	learn: 0.2107227	total: 33.5s	remaining: 1m 11s
318:	learn: 0.2104916	total: 33.6s	remaining: 1m 11s
319:	learn: 0.2103902	total: 33.7s	remaining: 1m 11s
320:	learn: 0.2103157	total: 33.8s	remaining: 1m 11s
321:	learn: 0.2103032	total: 33.8s	remaining: 1m 11s
322:	learn: 0.2102829	total: 33.9s	remaining: 1m 11s
323:	learn: 0.2102629	total: 34s	remaining: 1m 10s
324:	learn: 0.2100475	total: 34.1s	remaining: 1m 10s
325:	learn: 0.2099504	total: 34.2s	remaining: 1m 10s
326:	learn: 0.2099405	total: 34.3s	remaining: 1m 1

470:	learn: 0.1941507	total: 50s	remaining: 56.1s
471:	learn: 0.1940956	total: 50.1s	remaining: 56.1s
472:	learn: 0.1939605	total: 50.2s	remaining: 56s
473:	learn: 0.1938521	total: 50.3s	remaining: 55.8s
474:	learn: 0.1938055	total: 50.4s	remaining: 55.7s
475:	learn: 0.1937464	total: 50.6s	remaining: 55.8s
476:	learn: 0.1936718	total: 50.8s	remaining: 55.7s
477:	learn: 0.1936328	total: 50.9s	remaining: 55.6s
478:	learn: 0.1935127	total: 51s	remaining: 55.4s
479:	learn: 0.1932971	total: 51.1s	remaining: 55.3s
480:	learn: 0.1932157	total: 51.3s	remaining: 55.4s
481:	learn: 0.1931133	total: 51.4s	remaining: 55.3s
482:	learn: 0.1928235	total: 51.6s	remaining: 55.2s
483:	learn: 0.1926603	total: 51.6s	remaining: 55.1s
484:	learn: 0.1926251	total: 51.7s	remaining: 54.9s
485:	learn: 0.1924899	total: 51.9s	remaining: 54.9s
486:	learn: 0.1923773	total: 52s	remaining: 54.8s
487:	learn: 0.1923693	total: 52.1s	remaining: 54.6s
488:	learn: 0.1923170	total: 52.2s	remaining: 54.5s
489:	learn: 0.192148

632:	learn: 0.1777368	total: 1m 10s	remaining: 40.6s
633:	learn: 0.1775267	total: 1m 10s	remaining: 40.6s
634:	learn: 0.1775147	total: 1m 10s	remaining: 40.4s
635:	learn: 0.1774732	total: 1m 10s	remaining: 40.3s
636:	learn: 0.1773874	total: 1m 10s	remaining: 40.2s
637:	learn: 0.1772053	total: 1m 10s	remaining: 40.1s
638:	learn: 0.1771896	total: 1m 10s	remaining: 40s
639:	learn: 0.1770923	total: 1m 10s	remaining: 39.9s
640:	learn: 0.1770268	total: 1m 11s	remaining: 39.8s
641:	learn: 0.1769606	total: 1m 11s	remaining: 39.7s
642:	learn: 0.1768498	total: 1m 11s	remaining: 39.6s
643:	learn: 0.1768052	total: 1m 11s	remaining: 39.5s
644:	learn: 0.1767085	total: 1m 11s	remaining: 39.4s
645:	learn: 0.1766946	total: 1m 11s	remaining: 39.3s
646:	learn: 0.1766108	total: 1m 11s	remaining: 39.2s
647:	learn: 0.1764660	total: 1m 11s	remaining: 39.1s
648:	learn: 0.1764540	total: 1m 12s	remaining: 39s
649:	learn: 0.1764377	total: 1m 12s	remaining: 38.9s
650:	learn: 0.1764200	total: 1m 12s	remaining: 38.

788:	learn: 0.1671408	total: 1m 29s	remaining: 24s
789:	learn: 0.1671296	total: 1m 29s	remaining: 23.9s
790:	learn: 0.1670323	total: 1m 30s	remaining: 23.8s
791:	learn: 0.1669995	total: 1m 30s	remaining: 23.7s
792:	learn: 0.1669784	total: 1m 30s	remaining: 23.6s
793:	learn: 0.1669197	total: 1m 30s	remaining: 23.5s
794:	learn: 0.1668459	total: 1m 30s	remaining: 23.4s
795:	learn: 0.1665194	total: 1m 30s	remaining: 23.2s
796:	learn: 0.1664212	total: 1m 30s	remaining: 23.1s
797:	learn: 0.1663551	total: 1m 30s	remaining: 23s
798:	learn: 0.1661580	total: 1m 31s	remaining: 22.9s
799:	learn: 0.1660133	total: 1m 31s	remaining: 22.8s
800:	learn: 0.1659757	total: 1m 31s	remaining: 22.7s
801:	learn: 0.1659665	total: 1m 31s	remaining: 22.6s
802:	learn: 0.1659266	total: 1m 31s	remaining: 22.5s
803:	learn: 0.1659167	total: 1m 31s	remaining: 22.3s
804:	learn: 0.1659074	total: 1m 31s	remaining: 22.2s
805:	learn: 0.1659065	total: 1m 31s	remaining: 22.1s
806:	learn: 0.1658805	total: 1m 31s	remaining: 22s

946:	learn: 0.1582849	total: 1m 49s	remaining: 6.13s
947:	learn: 0.1582593	total: 1m 49s	remaining: 6.02s
948:	learn: 0.1581314	total: 1m 49s	remaining: 5.9s
949:	learn: 0.1581293	total: 1m 49s	remaining: 5.79s
950:	learn: 0.1581203	total: 1m 50s	remaining: 5.67s
951:	learn: 0.1581180	total: 1m 50s	remaining: 5.56s
952:	learn: 0.1581171	total: 1m 50s	remaining: 5.44s
953:	learn: 0.1581130	total: 1m 50s	remaining: 5.33s
954:	learn: 0.1580510	total: 1m 50s	remaining: 5.21s
955:	learn: 0.1580325	total: 1m 50s	remaining: 5.09s
956:	learn: 0.1579917	total: 1m 50s	remaining: 4.98s
957:	learn: 0.1579270	total: 1m 50s	remaining: 4.87s
958:	learn: 0.1579249	total: 1m 51s	remaining: 4.75s
959:	learn: 0.1579183	total: 1m 51s	remaining: 4.63s
960:	learn: 0.1578890	total: 1m 51s	remaining: 4.52s
961:	learn: 0.1578723	total: 1m 51s	remaining: 4.4s
962:	learn: 0.1578391	total: 1m 51s	remaining: 4.29s
963:	learn: 0.1576235	total: 1m 51s	remaining: 4.17s
964:	learn: 0.1575476	total: 1m 51s	remaining: 4

In [69]:
np.mean(cv_scores)

0.9068569468123465

In [70]:
%%time
ctb.fit(train_df_part, y_train_part,
        cat_features=cat_features_idx);

Learning rate set to 0.023211
0:	learn: 0.6667391	total: 166ms	remaining: 2m 46s
1:	learn: 0.6417144	total: 348ms	remaining: 2m 53s
2:	learn: 0.6176808	total: 500ms	remaining: 2m 46s
3:	learn: 0.5991836	total: 578ms	remaining: 2m 24s
4:	learn: 0.5804262	total: 690ms	remaining: 2m 17s
5:	learn: 0.5623603	total: 799ms	remaining: 2m 12s
6:	learn: 0.5450305	total: 1.05s	remaining: 2m 29s
7:	learn: 0.5280738	total: 1.19s	remaining: 2m 26s
8:	learn: 0.5135699	total: 1.35s	remaining: 2m 28s
9:	learn: 0.4974333	total: 1.46s	remaining: 2m 24s
10:	learn: 0.4823283	total: 1.6s	remaining: 2m 23s
11:	learn: 0.4711069	total: 1.82s	remaining: 2m 29s
12:	learn: 0.4587132	total: 1.94s	remaining: 2m 27s
13:	learn: 0.4481113	total: 2.01s	remaining: 2m 21s
14:	learn: 0.4360470	total: 2.17s	remaining: 2m 22s
15:	learn: 0.4263336	total: 2.34s	remaining: 2m 23s
16:	learn: 0.4163173	total: 2.41s	remaining: 2m 19s
17:	learn: 0.4089238	total: 2.54s	remaining: 2m 18s
18:	learn: 0.4008342	total: 2.61s	remaining: 

160:	learn: 0.2255656	total: 20.2s	remaining: 1m 45s
161:	learn: 0.2254571	total: 20.3s	remaining: 1m 45s
162:	learn: 0.2249788	total: 20.4s	remaining: 1m 44s
163:	learn: 0.2243620	total: 20.5s	remaining: 1m 44s
164:	learn: 0.2241075	total: 20.6s	remaining: 1m 44s
165:	learn: 0.2239205	total: 20.8s	remaining: 1m 44s
166:	learn: 0.2238685	total: 20.9s	remaining: 1m 44s
167:	learn: 0.2236417	total: 21s	remaining: 1m 44s
168:	learn: 0.2236061	total: 21.1s	remaining: 1m 43s
169:	learn: 0.2235823	total: 21.2s	remaining: 1m 43s
170:	learn: 0.2231857	total: 21.4s	remaining: 1m 43s
171:	learn: 0.2231576	total: 21.5s	remaining: 1m 43s
172:	learn: 0.2231576	total: 21.5s	remaining: 1m 42s
173:	learn: 0.2227503	total: 21.6s	remaining: 1m 42s
174:	learn: 0.2227147	total: 21.7s	remaining: 1m 42s
175:	learn: 0.2222462	total: 21.8s	remaining: 1m 42s
176:	learn: 0.2220806	total: 22s	remaining: 1m 42s
177:	learn: 0.2218220	total: 22.2s	remaining: 1m 42s
178:	learn: 0.2214780	total: 22.3s	remaining: 1m 4

318:	learn: 0.2054529	total: 38.6s	remaining: 1m 22s
319:	learn: 0.2053714	total: 38.7s	remaining: 1m 22s
320:	learn: 0.2053567	total: 38.8s	remaining: 1m 21s
321:	learn: 0.2052703	total: 38.8s	remaining: 1m 21s
322:	learn: 0.2052206	total: 38.9s	remaining: 1m 21s
323:	learn: 0.2050674	total: 39s	remaining: 1m 21s
324:	learn: 0.2049360	total: 39.2s	remaining: 1m 21s
325:	learn: 0.2048732	total: 39.3s	remaining: 1m 21s
326:	learn: 0.2048073	total: 39.4s	remaining: 1m 21s
327:	learn: 0.2047056	total: 39.5s	remaining: 1m 20s
328:	learn: 0.2047038	total: 39.6s	remaining: 1m 20s
329:	learn: 0.2046980	total: 39.7s	remaining: 1m 20s
330:	learn: 0.2046765	total: 39.8s	remaining: 1m 20s
331:	learn: 0.2045408	total: 39.9s	remaining: 1m 20s
332:	learn: 0.2044274	total: 40.1s	remaining: 1m 20s
333:	learn: 0.2042238	total: 40.2s	remaining: 1m 20s
334:	learn: 0.2041664	total: 40.3s	remaining: 1m 19s
335:	learn: 0.2040263	total: 40.5s	remaining: 1m 19s
336:	learn: 0.2037631	total: 40.6s	remaining: 1m

477:	learn: 0.1892345	total: 59.3s	remaining: 1m 4s
478:	learn: 0.1891592	total: 59.5s	remaining: 1m 4s
479:	learn: 0.1889328	total: 59.6s	remaining: 1m 4s
480:	learn: 0.1887868	total: 59.7s	remaining: 1m 4s
481:	learn: 0.1886278	total: 59.9s	remaining: 1m 4s
482:	learn: 0.1886120	total: 1m	remaining: 1m 4s
483:	learn: 0.1885403	total: 1m	remaining: 1m 4s
484:	learn: 0.1884694	total: 1m	remaining: 1m 4s
485:	learn: 0.1883639	total: 1m	remaining: 1m 4s
486:	learn: 0.1882396	total: 1m	remaining: 1m 3s
487:	learn: 0.1880801	total: 1m	remaining: 1m 3s
488:	learn: 0.1880305	total: 1m	remaining: 1m 3s
489:	learn: 0.1879218	total: 1m 1s	remaining: 1m 3s
490:	learn: 0.1878710	total: 1m 1s	remaining: 1m 3s
491:	learn: 0.1877798	total: 1m 1s	remaining: 1m 3s
492:	learn: 0.1876902	total: 1m 1s	remaining: 1m 3s
493:	learn: 0.1876863	total: 1m 1s	remaining: 1m 2s
494:	learn: 0.1875319	total: 1m 1s	remaining: 1m 2s
495:	learn: 0.1874552	total: 1m 1s	remaining: 1m 2s
496:	learn: 0.1874161	total: 1m 1

635:	learn: 0.1753275	total: 1m 23s	remaining: 47.6s
636:	learn: 0.1752893	total: 1m 23s	remaining: 47.4s
637:	learn: 0.1752715	total: 1m 23s	remaining: 47.3s
638:	learn: 0.1752544	total: 1m 23s	remaining: 47.2s
639:	learn: 0.1752295	total: 1m 23s	remaining: 47.1s
640:	learn: 0.1752055	total: 1m 23s	remaining: 46.9s
641:	learn: 0.1750050	total: 1m 23s	remaining: 46.8s
642:	learn: 0.1749184	total: 1m 24s	remaining: 46.7s
643:	learn: 0.1749054	total: 1m 24s	remaining: 46.6s
644:	learn: 0.1746882	total: 1m 24s	remaining: 46.5s
645:	learn: 0.1746216	total: 1m 24s	remaining: 46.3s
646:	learn: 0.1745236	total: 1m 24s	remaining: 46.2s
647:	learn: 0.1745060	total: 1m 24s	remaining: 46s
648:	learn: 0.1743530	total: 1m 24s	remaining: 46s
649:	learn: 0.1742963	total: 1m 25s	remaining: 45.8s
650:	learn: 0.1742138	total: 1m 25s	remaining: 45.7s
651:	learn: 0.1741767	total: 1m 25s	remaining: 45.6s
652:	learn: 0.1741388	total: 1m 25s	remaining: 45.5s
653:	learn: 0.1740983	total: 1m 25s	remaining: 45.

793:	learn: 0.1658795	total: 1m 46s	remaining: 27.5s
794:	learn: 0.1657523	total: 1m 46s	remaining: 27.4s
795:	learn: 0.1657396	total: 1m 46s	remaining: 27.3s
796:	learn: 0.1656758	total: 1m 46s	remaining: 27.1s
797:	learn: 0.1656603	total: 1m 46s	remaining: 27s
798:	learn: 0.1656167	total: 1m 46s	remaining: 26.9s
799:	learn: 0.1655328	total: 1m 47s	remaining: 26.8s
800:	learn: 0.1655276	total: 1m 47s	remaining: 26.6s
801:	learn: 0.1654226	total: 1m 47s	remaining: 26.5s
802:	learn: 0.1653817	total: 1m 47s	remaining: 26.4s
803:	learn: 0.1652069	total: 1m 47s	remaining: 26.2s
804:	learn: 0.1651573	total: 1m 47s	remaining: 26.1s
805:	learn: 0.1651329	total: 1m 48s	remaining: 26s
806:	learn: 0.1649416	total: 1m 48s	remaining: 25.9s
807:	learn: 0.1649222	total: 1m 48s	remaining: 25.7s
808:	learn: 0.1649169	total: 1m 48s	remaining: 25.6s
809:	learn: 0.1649045	total: 1m 48s	remaining: 25.5s
810:	learn: 0.1648398	total: 1m 48s	remaining: 25.4s
811:	learn: 0.1647995	total: 1m 48s	remaining: 25.

953:	learn: 0.1564483	total: 2m 10s	remaining: 6.27s
954:	learn: 0.1563562	total: 2m 10s	remaining: 6.14s
955:	learn: 0.1563332	total: 2m 10s	remaining: 6s
956:	learn: 0.1562703	total: 2m 10s	remaining: 5.86s
957:	learn: 0.1562672	total: 2m 10s	remaining: 5.73s
958:	learn: 0.1562365	total: 2m 10s	remaining: 5.59s
959:	learn: 0.1561399	total: 2m 10s	remaining: 5.45s
960:	learn: 0.1560936	total: 2m 11s	remaining: 5.32s
961:	learn: 0.1560792	total: 2m 11s	remaining: 5.19s
962:	learn: 0.1559846	total: 2m 11s	remaining: 5.05s
963:	learn: 0.1559179	total: 2m 11s	remaining: 4.91s
964:	learn: 0.1557985	total: 2m 11s	remaining: 4.78s
965:	learn: 0.1557577	total: 2m 11s	remaining: 4.64s
966:	learn: 0.1557453	total: 2m 12s	remaining: 4.5s
967:	learn: 0.1557193	total: 2m 12s	remaining: 4.37s
968:	learn: 0.1557064	total: 2m 12s	remaining: 4.23s
969:	learn: 0.1556855	total: 2m 12s	remaining: 4.09s
970:	learn: 0.1555947	total: 2m 12s	remaining: 3.96s
971:	learn: 0.1555481	total: 2m 12s	remaining: 3.8

In [71]:
roc_auc_score(y_valid, ctb.predict_proba(valid_df)[:, 1])

0.9169869126155554